### Módulo 5 Tarefa 1
#### Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

#### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

In [1]:
import pandas as pd
import requests

sinasc = pd.read_csv('SINASC_RO_2019.csv', sep=',', encoding='latin1')
print(sinasc.shape)

sinasc_sem_duplicatas = sinasc.drop_duplicates()
print(sinasc_sem_duplicatas.shape)

if sinasc.shape == sinasc_sem_duplicatas.shape:
    print('Não há duplicados na base de dados.')
else:
    print('Há duplicados na base de dados.')


(27028, 69)
(27028, 69)
Não há duplicados na base de dados.


In [2]:
display(sinasc_sem_duplicatas.head())

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,RondÃ´nia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,RondÃ´nia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,RondÃ´nia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,RondÃ´nia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,RondÃ´nia,-11.93554,-61.99982,338.0,7067.025


2. Conte o número de valores *missing* por variável. 

In [3]:
sinasc = pd.read_csv('SINASC_RO_2019.csv', sep=',', encoding='latin1')
missing_values_count = sinasc.isna().sum()

print(missing_values_count)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.

In [4]:
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc.loc[:, colunas_interesse]

missing_values_count = sinasc_selecionado.isna().sum()

print(missing_values_count)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.


In [5]:
sinasc_selecionado_sem_apgar5_faltante = sinasc_selecionado.dropna(subset=['APGAR5'])

num_linhas, num_colunas = sinasc_selecionado_sem_apgar5_faltante.shape
missing_values_count = sinasc_selecionado_sem_apgar5_faltante.isna().sum()

print(f'Número de linhas: {num_linhas}')
print(f'Número de colunas: {num_colunas}')
print(f'Número de valores faltantes em cada coluna:\n{missing_values_count}')

Número de linhas: 26925
Número de colunas: 9
Número de valores faltantes em cada coluna:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


5. Observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>

In [6]:
sinasc_selecionado['ESTCIVMAE'] = sinasc_selecionado['ESTCIVMAE'].fillna(9)
sinasc_selecionado['CONSULTAS'] = sinasc_selecionado['CONSULTAS'].fillna(9)

sinasc_selecionado_sem_apgar5_faltante = sinasc_selecionado.dropna(subset=['APGAR5'])

num_linhas, num_colunas = sinasc_selecionado_sem_apgar5_faltante.shape
missing_values_count = sinasc_selecionado_sem_apgar5_faltante.isna().sum()

print(f'Número de linhas: {num_linhas}')
print(f'Número de colunas: {num_colunas}')
print(f'Número de valores faltantes em cada coluna:\n{missing_values_count}')

Número de linhas: 26925
Número de colunas: 9
Número de valores faltantes em cada coluna:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero. 

In [7]:
sinasc_selecionado['QTDFILVIVO'] = sinasc_selecionado['QTDFILVIVO'].fillna(0)

sinasc_selecionado_sem_apgar5_faltante = sinasc_selecionado.dropna(subset=['APGAR5'])

num_linhas, num_colunas = sinasc_selecionado_sem_apgar5_faltante.shape
missing_values_count = sinasc_selecionado_sem_apgar5_faltante.isna().sum()

print(f'Número de linhas: {num_linhas}')
print(f'Número de colunas: {num_colunas}')
print(f'Número de valores faltantes em cada coluna:\n{missing_values_count}')

Número de linhas: 26925
Número de colunas: 9
Número de valores faltantes em cada coluna:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


7. Das restantes, decida que valores te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui. 


In [8]:
print(missing_values_count)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [9]:
sinasc_selecionado['ESCMAE'] = sinasc_selecionado['ESCMAE'].fillna(9)
sinasc_selecionado['GESTACAO'] = sinasc_selecionado['GESTACAO'].fillna(9)
sinasc_selecionado['GRAVIDEZ'] = sinasc_selecionado['GRAVIDEZ'].fillna(9)

sinasc_selecionado_sem_apgar5_faltante = sinasc_selecionado.dropna(subset=['APGAR5'])

num_linhas, num_colunas = sinasc_selecionado_sem_apgar5_faltante.shape
missing_values_count = sinasc_selecionado_sem_apgar5_faltante.isna().sum()

print(f'Número de linhas: {num_linhas}')
print(f'Número de colunas: {num_colunas}')
print(f'Número de valores faltantes em cada coluna:\n{missing_values_count}')




Número de linhas: 26925
Número de colunas: 9
Número de valores faltantes em cada coluna:
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>


In [13]:
bins = [0, 3, 5, 7, 10]
labels = ['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal']

sinasc_selecionado['APGAR5_CLASSIFICADO'] = pd.cut(sinasc_selecionado['APGAR5'], bins=bins, labels=labels, include_lowest=True)

print('Frequência de cada valor na categorização:')
print(sinasc_selecionado['APGAR5_CLASSIFICADO'].value_counts())

print('\nFrequência de cada valor na categorização em porcentagem:')
print(round(sinasc_selecionado['APGAR5_CLASSIFICADO'].value_counts() / len(sinasc_selecionado) * 100, 2))


Frequência de cada valor na categorização:
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: APGAR5_CLASSIFICADO, dtype: int64

Frequência de cada valor na categorização em porcentagem:
normal              97.91
asfixia leve         1.18
asfixia severa       0.27
asfixia moderada     0.25
Name: APGAR5_CLASSIFICADO, dtype: float64


9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [18]:
sinasc_selecionado.rename(columns={
    'ORIGEM': 'origem',
    'LOCNASC': 'local de nascimento',
    'ESTCIVMAE': 'estado civil da mãe',
    'IDADEMAE': 'idade_mae', 
    'ESCMAE': 'esc_mae', 
    'CODOCUPMAE': 'cod_ocup_mae', 
    'QTDFILVIVO': 'qtd_filhos_vivos', 
    'GESTACAO': 'semanas_gestacao', 
    'GRAVIDEZ': 'tipo_gravidez', 
    'PARTO': 'tipo_parto', 
    'APGAR1': 'apgar1', 
    'APGAR5': 'apgar5', 
    'SEXO': 'sexo', 
    'RACACOR': 'raca_cor', 
    'CONSULTAS': 'qtd_consultas', 
    'DTNASC': 'data_nascimento', 
    'APGAR5_CLASSIFICADO': 'apgar5_classificado'
}, inplace=True)


In [19]:
sinasc_selecionado

,local de nascimento,idade_mae,estado civil da mãe,esc_mae,qtd_filhos_vivos,semanas_gestacao,tipo_gravidez,qtd_consultas,apgar5,apgar5_classificado
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Ãnica,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Ãnica,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Ãnica,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Ãnica,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Ãnica,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Ãnica,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Ãnica,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Ãnica,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Ãnica,4,9.0,normal
